In [1]:
import warnings
warnings.filterwarnings('ignore')

# Speed comparison

## Introduction

The purpose of this page is to compare the speed of data transfer between the load method from this library: 

- [google_pandas_load.Loader.load()](Loader.rst#google_pandas_load.loader.Loader.load)

and Python Client for Google BigQuery’s methods: 

- [google.cloud.bigquery.job.QueryJob.to_dataframe()](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
- [google.cloud.bigquery.client.Client.load_table_from_dataframe()](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.client.Client.html#google.cloud.bigquery.client.Client.load_table_from_dataframe)

## Set up 

In [2]:
# %%bash 
# pip install google_pandas_load
# pip install google-cloud-bigquery-storage[pandas,fastavro]

In [3]:
import os
import pandas
from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta1
from google_pandas_load import LoaderQuickSetup

In [4]:
project_id = 'dmp-y-tests'
dataset_id = 'tmp'
bucket_name = 'bucket_gpl'
local_dir_path = '/tmp/gpl_directory'

Next, set bq_client, bqstorage_client, table_ref and instantiate a loader.

In [5]:
bq_client = bigquery.Client(
    project=project_id, 
    credentials=None)

bqstorage_client = bigquery_storage_v1beta1.BigQueryStorageClient(
    credentials=None)

table_ref = bigquery.dataset.DatasetReference(
    project=project_id, 
    dataset_id=dataset_id).table('s0')

gpl = LoaderQuickSetup(
    project_id=project_id, 
    dataset_id=dataset_id,
    bucket_name=bucket_name,
    local_dir_path=local_dir_path)

In [6]:
if not os.path.isdir(local_dir_path):
    os.makedirs(local_dir_path)

## Download

The query below creates a 580 MB BigQuery table.

In [7]:
query = """
select * from 
(select 'Hello, ' as a from unnest(generate_array(1, 6000))) 
cross join 
(select 'World!' as b from unnest(generate_array(1, 6000)))
"""

In [8]:
%%time
df = bq_client.query(query).to_dataframe()

CPU times: user 2min 57s, sys: 5.67 s, total: 3min 2s
Wall time: 15min


Using bqstorage_client speeds up the download. See [here](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas) for additional information.

In [9]:
%%time
df = bq_client.query(query).to_dataframe(bqstorage_client=bqstorage_client)

CPU times: user 1min 54s, sys: 12.2 s, total: 2min 6s
Wall time: 1min 52s


There is issue with the previous download: it used the cache query results !

In [10]:
%%time
job_config = bigquery.QueryJobConfig()
job_config.use_query_cache=False
df = bq_client.query(query, job_config=job_config).to_dataframe(bqstorage_client=bqstorage_client)

CPU times: user 1min 51s, sys: 11 s, total: 2min 2s
Wall time: 1min 57s


When executing a query with [google_pandas_load.Loader.load()](Loader.rst#google_pandas_load.loader.Loader.load), the query is not caught because the method always creates a destination table. See [here](https://cloud.google.com/bigquery/docs/cached-results) for more informations. 

In [11]:
%%time
df = gpl.load(source='query', destination='dataframe', query=query)

2019-11-25 11:38:48,596 - LoaderQuickSetup - DEBUG - Starting query to bq...
2019-11-25 11:39:03,433 - LoaderQuickSetup - DEBUG - Ended source to bq [14s, 0.0$]
2019-11-25 11:39:03,434 - LoaderQuickSetup - DEBUG - Starting bq to gs...
2019-11-25 11:39:12,630 - LoaderQuickSetup - DEBUG - Ended bq to gs [9s]
2019-11-25 11:39:12,632 - LoaderQuickSetup - DEBUG - Starting gs to local...
2019-11-25 11:39:14,332 - LoaderQuickSetup - DEBUG - Ended gs to local [1s]
2019-11-25 11:39:14,334 - LoaderQuickSetup - DEBUG - Starting local to dataframe...
2019-11-25 11:39:21,028 - LoaderQuickSetup - DEBUG - Ended local to dataframe [6s]


CPU times: user 6.84 s, sys: 363 ms, total: 7.2 s
Wall time: 32.6 s


## Upload

In [12]:
N = 36*10**6
df = pandas.DataFrame({'a': ['Hello, ']*N, 'b': ['World!']*N})

In [13]:
%%time
bq_client.load_table_from_dataframe(dataframe=df, destination=table_ref).result()

CPU times: user 5.72 s, sys: 324 ms, total: 6.04 s
Wall time: 1min 44s


In [14]:
%%time
df = gpl.load(source='dataframe', destination='bq', data_name='s1', dataframe=df)

2019-11-25 11:41:09,799 - LoaderQuickSetup - DEBUG - Starting dataframe to local...
2019-11-25 11:41:48,498 - LoaderQuickSetup - DEBUG - Ended dataframe to local [38s]
2019-11-25 11:41:48,499 - LoaderQuickSetup - DEBUG - Starting local to gs...
2019-11-25 11:41:48,894 - LoaderQuickSetup - DEBUG - Ended local to gs [0s]
2019-11-25 11:41:48,896 - LoaderQuickSetup - DEBUG - Starting gs to bq...
2019-11-25 11:42:43,310 - LoaderQuickSetup - DEBUG - Ended gs to bq [54s]


CPU times: user 39.5 s, sys: 293 ms, total: 39.8 s
Wall time: 1min 34s


## Conclusion 

The [google_pandas_load.Loader.load()](Loader.rst#google_pandas_load.loader.Loader.load) method executes faster downloads and faster uploads than those executed by the built-in methods from [Python Client for Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html). 

Looking at the [basic mechanism](index.rst#The-basic-mechanism), one could think that the use of the local folder diminishes the number of network calls, and thus it speeds up data transfers. 